In [1]:
#import libraries
import openai
import langchain
# import langchain-community
# import pypdf
import dotenv


In [2]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import openai
from langchain.vectorstores import pinecone
# from pinecone import Pinecone, ServerlessSpec

In [3]:
from dotenv import load_dotenv
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [4]:
import os

In [5]:
api_key = os.environ["OPENAI_API_KEY"]

In [6]:
# function to read the document
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    docs = file_loader.load()
    
    return docs    

In [7]:
all_docs = read_doc('documents')
len(all_docs)

60

In [8]:
# create a function to split the document
def chunk_creator(docs,chunk_size,chunk_overlap):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc = text_splitter.split_documents(docs)
    return doc

In [9]:
split_doc = chunk_creator(docs=all_docs,chunk_size=400,chunk_overlap=20)
split_doc

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-02-01T03:54:26+05:30', 'author': 'hss', 'moddate': '2025-02-01T03:56:01+05:30', 'title': '', 'source': 'documents\\budget_speech.pdf', 'total_pages': 60, 'page': 0, 'page_label': '1'}, page_content='GOVERNMENT OF INDIA\nBUDGET 2025-2026\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1,  2025'),
 Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-02-01T03:54:26+05:30', 'author': 'hss', 'moddate': '2025-02-01T03:56:01+05:30', 'title': '', 'source': 'documents\\budget_speech.pdf', 'total_pages': 60, 'page': 2, 'page_label': '3'}, page_content='CONTENTS \n \nPART – A \n Page No. \nIntroduction 1 \nBudget Theme 1 \nAgriculture as the 1st engine 3 \nMSMEs as the 2nd engine 6 \nInvestment as the 3rd engine 8 \nA. Investing in People 8 \nB. Investing in the Economy 10 \nC. Investing in Innovation 14 \nExpor

In [10]:
len(split_doc)

275

In [11]:
from langchain_openai import OpenAIEmbeddings

In [12]:
# pip install google-genai

In [13]:
from google import genai

In [14]:
# generate ollama/open ai/google AI embeddings
openai_embeddings = OpenAIEmbeddings(api_key=api_key,model="text-embedding-3-large")
openai_embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000019E855C5880>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000019E85E307D0>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [15]:
# creating google gemini embeddings as OPEN AI is not free
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])
google_embeddings = client.models.embed_content(
        model="gemini-embedding-exp-03-07",
        contents="What is the meaning of life?")
google_embeddings

EmbedContentResponse(embeddings=[ContentEmbedding(values=[-0.022372285, -0.004451784, 0.013473644, -0.053762246, -0.020569915, 0.011864573, 0.015185799, 0.006950965, 0.03180835, 0.007074574, 0.027503368, -0.00600613, -0.014889315, 0.03269886, 0.12054204, 0.019322146, 0.000517173, 0.0045754807, -0.00856155, -0.01532448, 0.015616342, -0.008661197, -0.017454486, 0.0099245, -0.015551475, 0.012284064, 0.020809751, -0.0037114064, 0.025106275, 0.008105811, 0.020252233, 0.0019548477, -0.010780675, 0.027334962, -0.017213175, -0.011735542, 0.009507163, -0.015499457, -0.013591795, 0.0138707, -0.022853972, -0.009638755, -0.0034423112, -0.018855078, 0.018475225, -0.010515843, 0.015031793, -0.042978574, -0.013993226, 0.007916359, -0.012274015, 0.011758872, -0.010251529, -0.15881006, 0.016281825, 0.0103672175, -0.006364179, -0.009997806, -0.025991082, -0.027687864, -0.008586312, -0.014933809, -0.007584574, -0.021427568, 0.008805106, -0.009369353, -0.02012641, 0.011695516, 0.0020037016, 0.012606018, -

In [16]:
# pip install langchain-ollama

In [17]:
from langchain_ollama import OllamaEmbeddings

In [18]:
ollama_embedding = OllamaEmbeddings(model="nomic-embed-text")

In [19]:
vectors = ollama_embedding.embed_query("Hello!")
len(vectors)

768

In [1]:
pip show pinecone

Name: pinecone
Version: 7.0.0
Summary: Pinecone client and SDK
Home-page: 
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: c:\Python312\Lib\site-packages
Requires: certifi, pinecone-plugin-interface, python-dateutil, typing-extensions, urllib3
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
from pinecone import Pinecone, ServerlessSpec

ImportError: cannot import name 'Pinecone' from 'pinecone' (unknown location)

In [ ]:
# creating vector DB for chat/response storage
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

NameError: name 'Pinecone' is not defined